In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("biourja-input.csv")

In [4]:
df.columns

Index(['Plant_Name', 'Forecast', 'Capacity'], dtype='object')

In [15]:
# Total forecast
df["State Forecast"] = 12000

# Zone wise forecast
df["Zone"] = ""
df.loc[0:24, 'Zone'] = "East"
df.loc[24:50, 'Zone'] = "North"
df.loc[50:65, 'Zone'] = "West"
df.loc[65:101, 'Zone'] = "South"

df["Zonal Forecast"] = 0
df.loc[0:24, 'Zonal Forecast'] = 2800
df.loc[24:50, 'Zonal Forecast'] = 1500
df.loc[50:65, 'Zonal Forecast'] = 2000
df.loc[65:101, 'Zonal Forecast'] = 6500

# Summation of zonal forecast while removing the duplicates
df["Total Zonal Forecast"]=df['Zonal Forecast'].drop_duplicates().sum()
# Finding the ratio of zonal forecast/Total zonal forecast for finding the weights in which particular zone must be reduced
df["Ratio_Zonal_TotalZonal"]=df["Zonal Forecast"]/df["Total Zonal Forecast"]

# Finding the actual value to be reduced from each zonal forecast
df["How Much To Reduce From Total"]=df["Ratio_Zonal_TotalZonal"]*abs(df["State Forecast"]-df["Total Zonal Forecast"])

# Finding ratio of forecast/capacity
df["Ratio_Forecast_Capacity"]=df["Forecast"]/df["Capacity"]

In [6]:
# Summation of each Zonal Forecast
sum_east = df.iloc[0:24]['Ratio_Forecast_Capacity'].sum()
sum_north = df.iloc[24:50]['Ratio_Forecast_Capacity'].sum()
sum_west = df.iloc[50:65]['Ratio_Forecast_Capacity'].sum()
sum_south = df.iloc[65:101]['Ratio_Forecast_Capacity'].sum()


# Normalization of each zonal forecast total
df["Norm_Ratio_Forecast_Capacity"] = 0 
df.loc[0:24, 'Norm_Ratio_Forecast_Capacity'] = df.loc[0:24, 'Ratio_Forecast_Capacity'] / sum_east
df.loc[24:50, 'Norm_Ratio_Forecast_Capacity'] = df.loc[24:50, 'Ratio_Forecast_Capacity'] / sum_north
df.loc[50:65, 'Norm_Ratio_Forecast_Capacity'] = df.loc[50:65, 'Ratio_Forecast_Capacity'] / sum_west
df.loc[65:101, 'Norm_Ratio_Forecast_Capacity'] = df.loc[65:101, 'Ratio_Forecast_Capacity'] / sum_south

C:\Users\HP\AppData\Local\Temp\ipykernel_8648\2796918006.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.03050699 0.01397677 0.01142332 0.05926689 0.04408059 0.05342084
 0.04488694 0.05169614 0.04972506 0.05268168 0.05641106 0.03487474
 0.05362243 0.01404397 0.06020764 0.04690282 0.01572387 0.00752595
 0.06473217 0.05949088 0.06544893 0.01303603 0.03870491 0.05760939
 0.05661265]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[0:24, 'Norm_Ratio_Forecast_Capacity'] = df.loc[0:24, 'Ratio_Forecast_Capacity'] / sum_east


Sachin

In [7]:
#Finding Actual value to reduce from each plant 
df["How To Reduce From Each Plant"]=df["Norm_Ratio_Forecast_Capacity"]*df["How Much To Reduce From Total"]

In [8]:
# Determination of new Forecast
df["New Forecast"]=round(df["Forecast"]-df["How To Reduce From Each Plant"], 1)

In [9]:
df

,Plant_Name,Forecast,Capacity,State Forecast,Zone,Zonal Forecast,Total Zonal Forecast,Ratio_Zonal_TotalZonal,How Much To Reduce From Total,Ratio_Forecast_Capacity,Norm_Ratio_Forecast_Capacity,How To Reduce From Each Plant,New Forecast
0,E1,45.4,100,12000,East,2800,12800,0.218750,175.00,0.454000,0.030507,5.338724,40.1
1,E2,31.2,150,12000,East,2800,12800,0.218750,175.00,0.208000,0.013977,2.445935,28.8
2,E3,8.5,50,12000,East,2800,12800,0.218750,175.00,0.170000,0.011423,1.999082,6.5
3,E4,88.2,100,12000,East,2800,12800,0.218750,175.00,0.882000,0.059267,10.371706,77.8
4,E5,98.4,150,12000,East,2800,12800,0.218750,175.00,0.656000,0.044081,7.714103,90.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,S31,145.4,250,12000,South,6500,12800,0.507812,406.25,0.581600,0.024828,10.086517,135.3
96,S32,121.0,150,12000,South,6500,12800,0.507812,406.25,0.806667,0.034436,13.989782,107.0
97,S33,181.6,200,12000,South,6500,12800,0.507812,406.25,0.908000,0.038762,15.747176,165.9
98,S34,25.4,50,12000,South,6500,12800,0.507812,406.25,0.508000,0.021686,8.810094,16.6


In [14]:
# Saving to outputcsv (only plant name and new forecast)
df.to_csv("biourja-output.csv", index=False, columns={"Plant_Name", "New Forecast"})